In [ ]:
import os
import pandas as pd

In [ ]:
import pandas as pd
import os

# Input directory where your stock datasets are stored
input_directory = '/content/processed_files'

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through all CSV files in the directory
for filename in os.listdir(input_directory):
    if filename.endswith('.csv'):  # Ensure only CSV files are selected
        file_path = os.path.join(input_directory, filename)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)


        # Append the DataFrame to the list
        dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dataframes, ignore_index=True)

# Display the concatenated DataFrame
print(final_df.head())

# Save the concatenated DataFrame to a new CSV file
final_df.to_csv('/content/Stocks2.csv', index=False)
print("All datasets combined and saved as 'all_stocks_combined.csv'")


<ipython-input-8-c70e19b39a4c>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(dataframes, ignore_index=True)


     S.N. Symbol        Date    LTP   High    Low   Qty  Turnover  SMA_5  \
0  2377.0  SADBL  2012-09-25  147.0  152.0  147.0    30    4480.0  169.0   
1  2376.0  SADBL  2012-09-26  143.0  145.0  143.0    70   10030.0  159.6   
2  2375.0  SADBL  2012-09-27  137.0  141.0  137.0    60    8340.0  150.8   
3  2374.0  SADBL  2012-10-02  124.0  135.0  124.0   200   25460.0  141.2   
4  2373.0  SADBL  2012-10-03  112.0  122.0  112.0  4850  544740.0  132.6   

    returns  ...       MACD  Volatility  Volume_MA5  Volume_MA20  month  \
0 -0.051613  ... -36.547462    0.025549     73404.0      64561.5      9   
1 -0.027211  ... -37.781527    0.025759     56410.0      64906.0      9   
2 -0.041958  ... -38.796462    0.025648     10294.0      64553.0      9   
3 -0.094891  ... -40.186551    0.030081     11602.0      64214.5     10   
4 -0.096774  ... -41.774950    0.032350    118610.0      88919.5     10   

   day_of_week day_of_month week_of_year days_from_start High   
0            1           25

In [ ]:
# Directory where your CSV files are located
input_directory = '/content/'
# Directory where you want to save the processed files
output_directory = '/content/processed_files/'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Preprocessing function
def preprocess_file(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Convert numeric columns to proper format
    numeric_columns = ['Qty', 'Turnover', 'LTP', 'High', 'Low']
    for col in numeric_columns:
        if col in df.columns:  # Check if the column exists
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

    # Convert 'Date' column to datetime and clean the data
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.sort_values(by='Date')
        df = df.drop_duplicates(subset='Date', keep='first').reset_index(drop=True)

    # Add technical indicators
    if 'LTP' in df.columns and 'Turnover' in df.columns:
        df['SMA_5'] = df['LTP'].rolling(window=5).mean()
        df['returns'] = df['LTP'].pct_change()
        df['SMA_20'] = df['LTP'].rolling(window=20).mean()
        delta = df['LTP'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        EMA_12 = df['LTP'].ewm(span=12, adjust=False).mean()
        EMA_26 = df['LTP'].ewm(span=26, adjust=False).mean()
        df['MACD'] = EMA_12 - EMA_26
        df['Volatility'] = df['returns'].rolling(window=10).std()
        df['Volume_MA5'] = df['Turnover'].rolling(window=5).mean()
        df['Volume_MA20'] = df['Turnover'].rolling(window=20).mean()

    # Drop rows with NaN values (optional)
    df = df.dropna()

    df['month'] = df['Date'].dt.month.astype("str").astype('category')
    df['day_of_week'] = df['Date'].dt.dayofweek.astype("str").astype('category')
    df['day_of_month'] = df['Date'].dt.day.astype("str").astype('category')
    df['week_of_year'] = df['Date'].dt.isocalendar().week
    df['days_from_start'] = (df['Date'] - df['Date'].min()).dt.days


    return df

# Process each CSV file in the /content/ directory
for filename in os.listdir(input_directory):
    if filename.endswith('.csv'):  # Process only .csv files
        input_path = os.path.join(input_directory, filename)
        output_path = os.path.join(output_directory, filename)

        # Preprocess the file
        print(f"Processing {filename}...")
        processed_df = preprocess_file(input_path)

        # Save the processed file to the output directory
        processed_df.to_csv(output_path, index=False)
        print(f"Saved processed file to {output_path}")

print(f"All files have been processed and saved to {output_directory}")

Processing LSL.csv...
Saved processed file to /content/processed_files/LSL.csv
Processing NABBC_stock_history.csv...
Saved processed file to /content/processed_files/NABBC_stock_history.csv
Processing FinalNIMB.csv...
Saved processed file to /content/processed_files/FinalNIMB.csv
Processing GRDBL.csv...
Saved processed file to /content/processed_files/GRDBL.csv
All files have been processed and saved to /content/processed_files/
